Authors: Jagat Kafle, Yerik Singh
Course: Oscillations, Waves, and Optics (2022)

This jupyter notebook document is a merged document which solves two dimensional wave equation with constant velocity and variable velocity. The title is given before the start of each instance.

# The 2D Wave Equation with constant velocity

In [ ]:
# importing the libraries
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

In [ ]:
# L - Variable for size (in the x-axis)
# N - Number of grid box (in the x-axis)
# Dt - Time step
# Nt - End time 
# aspect - ratio between the horizontal and the veritcal
# wave2D - solves two dimensional wave equation and returns u_t, X, Y, delta, Dt, step

def wave2D(L=0.5, N=50, Dt=1e-5, Nt=1500, step=1, aspect=1):

    Lx = L # Length of the horizontal component of the grid
    Ly = aspect*L # Length of the vertical component of the grid
    Nx = N # Number of grid boxes in x
    Ny = int(aspect*N) 
    delta = L/N # the grid spacing step


    c = 500 # Speed of the wave

    C = c*Dt/delta # Courant number
    Csq = C**2 # The sq of the Courant number

    # the points in the meshgrid
    x = np.linspace(0, Lx, Nx+1)  # we need N+1 points for N spacing
    y = np.linspace(0, Ly, Ny+1)  # we need N+1 points for N spacing
    # note: we use u[i*delta, j*delta] = u(x, y)
    # (which is the transpose of the numpy row-column layout so we transpose X and Y)
    X, Y = np.meshgrid(x, y)
    X, Y = X.T, Y.T
   
   # gaussian - set up the initial form of the wave
    def gaussian(x, um=0.05, x0=None, sigma=0.1*L):
        x0 = np.mean(x) if x0 is None else x0
        g = um/np.sqrt(2*np.pi*sigma**2) * np.exp(-(x-x0)**2 / (2*sigma**2))
        return g
    
    #def sin(x):
    #return np.sin(x)
    
    #Creates a two dimensional array for u_{-1}, u, u_{+1}
    um = np.zeros((Nx+1, Ny+1))
    u = np.zeros_like(um)
    up = np.zeros_like(um)
    #H = np.zeros_like(um)
    # Three dimensional array that stores, Time, X points, Y points
    u_t = np.zeros((int(np.ceil(Nt/step)), Nx+1, Ny+1)) 

    # Used to determine whether the point under consideration is on the boundary or inside it. 
    # If it is inside the bundary - True
    # If it is on the boundary - False
    inside = np.zeros_like(um, dtype= bool)
    inside[1:-1, 1:-1] = True
    boundary = np.logical_not(inside)
    
    # Establishing boundary conditions
    def set_boundary_conditions(u):
        u[boundary] = 0 # Anything that is on the boundary is equated to 0.
  
    # Boundaries of um, u, and up is taken as 0.
    set_boundary_conditions(um)
    set_boundary_conditions(u)
    set_boundary_conditions(up)

    # Initially there is no difference in the inside of um and u.
    # The shape of both of them is set up by the guassian in a symmetric manner.
    um[inside] = u[inside] = (gaussian(X)*gaussian(Y))[inside]
    #um[inside] = u[inside] = (np.sin(2*np.pi*X)*np.sin(2*np.pi*Y))[inside]
    #um[inside] = u[inside] = (X**2*Y)[inside]
    
    # initialising
    t_index = 0
    u_t[t_index, :, :] = um
    if step == 1:
        t_index += 1
        u_t[t_index, :, :] = u_t[t_index,:,:] + 50*Dt
    
#     def H(x,y):
#         return np.sin(x)*np.sin(y)
        
    # vectorized form to calculate the wave in next time step
    for nt in range(2, Nt):
        up[1:-1, 1:-1] = 2*(1-2*Csq)*u[1:-1, 1:-1] - um[1:-1, 1:-1] + Csq*(u[2:, 1:-1] 
                                            + u[:-2, 1:-1] + u[1:-1, 2:] + u[1:-1, :-2])
        um[:, :], u[:, :] = u, up

        if nt % step == 0 or nt == Nt-1:
            t_index += 1
            u_t[t_index, :, :] = up       
            print("Iteration {0:5d}".format(nt), end="\r")
 
    return u_t, X, Y, delta, Dt, step

In [ ]:
# Running the solver function
u_t, X, Y, delta, Dt, step = wave2D(Nt=5000)

In [ ]:
# 3d wireframe plot
def plot_wireframe(X, Y, u, limits=None, ax=None):
    if ax is None:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection="3d")
    ax.plot_wireframe(X, Y, u, color='blue')
    ax.set_xlabel(r"position $x$ (m)")
    ax.set_ylabel(r"position $y$ (m)")
    ax.set_zlabel(r"displacement $u$ (m)")
    ax.set_zlim(limits)
    ax.figure.tight_layout()
    return ax

In [ ]:
# 3d surface plot
def plot_surface(X, Y, u, limits=None, ax=None):
    if ax is None:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection="3d")
    ax.plot_surface(X, Y, u, cmap='jet')
    ax.set_xlabel(r"position $x$")
    ax.set_ylabel(r"position $y$")
    ax.set_zlabel(r"displacement $u$")
    ax.set_zlim(limits)
    ax.figure.tight_layout()
    return ax

In [ ]:
#plot the surface
for i in range(0, 100, 20):
    plot_surface(X, Y, u_t[i], limits=(-0.5,0.5))

In [ ]:
# Function to return an animation, given a 2D array like u[i][j] that we have created.
# Notice how I have defined many default options that can be used to download the animation as a video
def animate(array, save_animation=False, save_name='video.mp4', save_dpi=300, save_fps=100):
    
    fig, ax = plt.subplots()                    # Define a figure and an axes. axes works just like plt, i.e. you can just do ax.plot, etc.
    
    #graph, = ax.plot([], [], color='firebrick',lw=3) # Define an ``empty'' plot, we will be dynamically changing the data in this plot
    im = ax.imshow(array[0])

    def init():                                                           # Initialise the whole plot
        # ax.set_xlim(0,1)                            # Setting the x and y limits of the plot
        # ax.set_ylim(-1,1)
    
        # ax.tick_params(top=False, bottom=False, left=False, right=False,  # Remove all axes (they're unnecessary here)
        #                labelleft=False, labelbottom=False)
        return [im]
    
    def animate(frame):                 # For frame number `frame',
        # ydata = array[frame]            # for every time-step, get the string's position data, and
        # graph.set_data(x, ydata)        # plot the current position data against x (i.e., set the data for the graph object)
        im.set_data(array[frame])
        return [im]
        #return graph                    

    # Most important line, this is what actually handles the animation by calling the `animate' function with a frame number (integer)
    ani = FuncAnimation(fig, animate, init_func=init, blit=True, frames=1000, interval=20, repeat=True)   # Code to create animations
    
    if(save_animation):                                  # If you wish to save the animation, do so with the dpi and fps set here 
        ani.save(save_name, dpi=save_dpi, fps=save_fps)

    return ani                                           # When calling this FuncAnimation function from with a code, all the information that update the window 
                                                         # are attributes of the object ani. If you do not keep a reference to it around, then ani is garbage collected 
                                                         # all information about the graphs disappears when calling from within a function. You don't need this when you call 
                                                         # aren't calling the animation from within a function.

In [ ]:
# %matplotlib notebook
# animate(u_t)

In [ ]:
# for i in range(0,51):
#     plt.scatter(i, (np.sum(u_t[i]**2*delta**2)/np.sum(u_t[0]**2*delta**2)))

In [ ]:
# Function to return an animation, given a 2D array like u[i][j] that we have created.
# Notice how I have defined many default options that can be used to download the animation as a video
def animate_3d(array, save_animation=False, save_name='video.mp4', save_dpi=300, save_fps=100):
    
#     fig, ax = plt.subplots()                    # Define a figure and an axes. axes works just like plt, i.e. you can just do ax.plot, etc.
    
    #graph, = ax.plot([], [], color='firebrick',lw=3) # Define an ``empty'' plot, we will be dynamically changing the data in this plot
#     im = ax.imshow(array[0])
    fig = plt.figure()
    ax = fig.add_subplot(111,projection='3d')
    
    #M,N = np.meshgrid(X,Y)
    def init():                                                           # Initialise the whole plot
        ax.plot_wireframe(X,Y,array[0])
        ax.set_zlim(-0.25,0.25)
        return [ax]
        # ax.tick_params(top=False, bottom=False, left=False, right=False,  # Remove all axes (they're unnecessary here)
        #                labelleft=False, labelbottom=False)
#         return [im]
    
    def animate(frame):                 # For frame number `frame',
        # ydata = array[frame]            # for every time-step, get the string's position data, and
        # graph.set_data(x, ydata)        # plot the current position data against x (i.e., set the data for the graph object)
        plt.cla()
        #ax.plot_wireframe(X,Y,array[frame])
        ax.plot_surface(X,Y,array[frame], cmap='jet')
        ax.set_xlabel('x-axis')
        ax.set_ylabel('y-axis')
        ax.set_zlabel('z-axis')
        ax.set_zlim(-0.25,0.25)
        return [ax]
        #return graph                    

    # Most important line, this is what actually handles the animation by calling the `animate' function with a frame number (integer)
    ani = FuncAnimation(fig, animate, init_func=init, blit=True, frames=1000, interval=20, repeat=True)   # Code to create animations
    
    if(save_animation):                                  # If you wish to save the animation, do so with the dpi and fps set here 
        ani.save(save_name, dpi=save_dpi, fps=save_fps)

    return ani                                           # When calling this FuncAnimation function from with a code, all the information that update the window 
                                                         # are attributes of the object ani. If you do not keep a reference to it around, then ani is garbage collected 
                                                         # all information about the graphs disappears when calling from within a function. You don't need this when you call 
                                                         # aren't calling the animation from within a function.

In [ ]:
%matplotlib notebook
animate_3d(u_t)

In [ ]:
def amplitude(u_t, a, b):
    u = []
    for i in range(0, 1000):
        u.append(u_t[i][a][b])
    return u

# for i in range(0, 1000):
#     u.append(u_t[i][50][20])
#plt.plot(amplitude(u_t, 10, 10), color='red', linewidth=2)
#plt.plot(amplitude(u_t, 20, 15), color='red', linewidth=2)
#plt.plot(amplitude(u_t, 20, 30), color='red', linewidth=2)
plt.plot(amplitude(u_t, 40, 40), color='red', linewidth=2)
plt.xlabel('time index')
plt.ylabel('amplitude')
#plt.xlim(5, 1000)

# The 2D Tsunami Model with variable velocity

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

In [ ]:
# L - Variable for size (in the x-axis)
# N - Number of grid box (in the x-axis)
# Dt - Time step
# Nt - End time 
# aspect - ratio between the horizontal and the veritcal
# wave2D - solves two dimensional wave equation and returns u_t, X, Y, delta, Dt, step

def wave2D(L=0.5, N=100, Dt=1e-4, Nt=1500, step=1, aspect=1):

    Lx = L # Length of the horizontal component of the grid
    Ly = aspect*L # Length of the vertical component of the grid
    Nx = N # Number of grid boxes in x
    Ny = int(aspect*N) 
    delta = L/N # the grid spacing step

    c = 50 # Speed of the wave

    C = c*Dt/delta # Courant number
    Csq = C**2 # The sq of the Courant number
    m = Dt/delta
    msq = m**2
    # the points in the meshgrid
    x = np.linspace(0, Lx, Nx+1)  # we need N+1 points for N spacing
    y = np.linspace(0, Ly, Ny+1)  # we need N+1 points for N spacing
    # note: we use u[i*delta, j*delta] = u(x, y)
    # (which is the transpose of the numpy row-column layout so we transpose X and Y)
    X, Y = np.meshgrid(x, y)
    X, Y = X.T, Y.T
   
   # gaussian - set up the initial form of the wave
    def gaussian(x, um=0.05, x0=None, sigma=0.1*L):
        x0 = np.mean(x) if x0 is None else x0
        g = um/np.sqrt(2*np.pi*sigma**2) * np.exp(-(x-x0)**2 / (2*sigma**2))
        return g
    
    #def sin(x):
    #return np.sin(x)
    
    # create the water depth array
    h = np.zeros(Nx+1)
    g = np.zeros_like(h)
    for i in range(Nx+1):
        h[i:] = - x[i]
    g = 9.8*h
    #Creates a two dimensional array for u_{-1}, u, u_{+1}
    um = np.zeros((Nx+1, Ny+1))
    u = np.zeros_like(um)
    up = np.zeros_like(um)
    base = np.zeros_like(um)
    
    for i in range(Nx):
        for j in range(Ny):
            base[i, j] = h[i]
    
    #H = np.zeros_like(um)
    # Three dimensional array that stores, Time, X points, Y points
    u_t = np.zeros((int(np.ceil(Nt/step)), Nx+1, Ny+1)) 

    # Used to determine whether the point under consideration is on the boundary or inside it. 
    # If it is inside the bundary - True
    # If it is on the boundary - False
    inside = np.zeros_like(um, dtype= bool)
    inside[0:-1, 0:-1] = True
    boundary = np.logical_not(inside)
    
    # Establishing boundary conditions
    def set_boundary_conditions(u):
        u[boundary] = 0
#     def set_boundary_conditions(u, um, up):
#         j = 0
#         while j < 51:
#             up[1, j] = um[i, j] + msq*((0.5*(g[2:] + g[1:-1])*(u[2:, 1:-1] - u[1:-1, 1:-1]) - 0.5*(g[1:-1] + g[:-2])*(u[1:-1, 1:-1]-u[:-2, 1:-1]))+ g[1:-1]*(u[1:-1,2:] - 2*u[1:-1, 1:-1] + u[1:-1, :-2]))
#             j = j + 1
    
    # Boundaries of um, u, and up is taken as 0.
    set_boundary_conditions(um)
    set_boundary_conditions(u)
    set_boundary_conditions(up)

    # Initially there is no difference in the inside of um and u.
    # The shape of both of them is set up by the guassian in a symmetric manner.
    #um[inside] = u[inside] = (1.5*gaussian(3*(X))*gaussian(Y))[inside]
    um[inside] = u[inside] = (gaussian(X)*gaussian(Y))[inside]
    #um[inside] = u[inside] = (np.sin(2*np.pi*X)*np.sin(2*np.pi*Y))[inside]
    #um[inside] = u[inside] = ((X)**2*(Y))[inside]
    #um[inside] = u[inside] = ((X)**2)[inside]
    
    # initialising
    t_index = 0
    u_t[t_index, :, :] = um
    if step == 1:
        t_index += 1
        u_t[t_index, :, :] = u_t[0,:,:] + 50*Dt
    
#     def H(x,y):
#         return np.sin(x)*np.sin(y)    
    for nt in range(2, Nt):
        up[i, j] = 2*u[i, j] 
    # vectorized form to calculate the wave in next time step
    for nt in range(2, Nt):
        #up[1:-1, 1:-1] = 2*(1-2*Csq)*u[1:-1, 1:-1] - um[1:-1, 1:-1] + Csq*(u[2:, 1:-1] 
                                              #+ u[:-2, 1:-1] + u[1:-1, 2:] + u[1:-1, :-2]) + Csq*((0.5*(h[2:] + h[1:-1])*(u[2:, 1:-1] - u[1:-1, 1:-1]) - 0.5*(h[1:-1] + h[:-2])*(u[1:-1, 1:-1]-u[:-2, 1:-1]))+ h[1:-1]*(u[1:-1,2:] - 2*u[1:-1, 1:-1] + u[1:-1, :-2]))
        #up[1:-1, 1:-1] = 2*u[1:-1,1:-1] - um[1:-1, 1:-1] + msq*((0.5*(h[2:] + h[1:-1])*(u[2:, 1:-1] - u[1:-1, 1:-1]) - 0.5*(h[1:-1] + h[:-2])*(u[1:-1, 1:-1]-u[:-2, 1:-1]))+ h[1:-1]*(u[1:-1,2:] - 2*u[1:-1, 1:-1] + u[1:-1, :-2]))
        up[1:-1, 1:] = 2*u[1:-1,1:] - um[1:-1, 1:] + msq*((0.5*(g[1:] + g[0:-1])*(u[2:, 1:] - u[1:-1, 1:]) - 0.5*(g[0:-1] + g[1:])*(u[1:-1, 1:]-u[:-2, 1:]))+ g[1:]*(u[1:-1,1:] - 2*u[1:-1, 1:] + u[1:-1, :-1]))
        um[:, :], u[:, :] = u, up

        if nt % step == 0 or nt == Nt-1:
            t_index += 1
            u_t[t_index, :, :] = up       
            print("Iteration {0:5d}".format(nt), end="\r")
 
    return u_t, X, Y, delta, Dt, step, h,x, y, base

In [ ]:
#Running the solver function
u_t, X, Y, delta, Dt, step, h, x, y, base= wave2D(Nt=1000)

In [ ]:
# 3d wireframe plot
def plot_wireframe(X, Y, u, limits=None, ax=None, color=None):
    if ax is None:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection="3d")
    ax.plot_wireframe(X, Y, u, color='blue')
    ax.set_xlabel(r"position $x$ (m)")
    ax.set_ylabel(r"position $y$ (m)")
    ax.set_zlabel(r"displacement $u$ (m)")
    ax.set_zlim(limits)
    ax.figure.tight_layout()
    return ax

In [ ]:
# 3d surface plot
def plot_surface(X, Y, u, limits=None, ax=None):
    if ax is None:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection="3d")
    ax.plot_surface(X, Y, u, cmap='jet')
    ax.set_xlabel(r"position $x$")
    ax.set_ylabel(r"position $y$")
    ax.set_zlabel(r"displacement $u$")
    ax.set_zlim(limits)
    ax.figure.tight_layout()
    return ax

In [ ]:
# # plot the water surface
for i in range(0, 300, 50):
    plot_surface(X, Y, u_t[i], limits=(-0.5,0.5))

In [ ]:
# Function to return an animation, given a 2D array like u[i][j] that we have created.
# Notice how I have defined many default options that can be used to download the animation as a video
def animate_3d(array, save_animation=False, save_name='video.mp4', save_dpi=300, save_fps=100):
    
#     fig, ax = plt.subplots()                    # Define a figure and an axes. axes works just like plt, i.e. you can just do ax.plot, etc.
    
    #graph, = ax.plot([], [], color='firebrick',lw=3) # Define an ``empty'' plot, we will be dynamically changing the data in this plot
#     im = ax.imshow(array[0])
    fig = plt.figure()
    ax = fig.add_subplot(111,projection='3d')
    
    #M,N = np.meshgrid(X,Y)
    def init():                                                           # Initialise the whole plot
        
        ax.plot_wireframe(X,Y,array[0])
        ax.set_zlim(-0.5,0.5)
        return [ax]
        # ax.tick_params(top=False, bottom=False, left=False, right=False,  # Remove all axes (they're unnecessary here)
        #                labelleft=False, labelbottom=False)
        #return [im]
    
    def animate(frame):                 # For frame number `frame',
        # ydata = array[frame]            # for every time-step, get the string's position data, and
        # graph.set_data(x, ydata)        # plot the current position data against x (i.e., set the data for the graph object)
        
        plt.cla()
        #ax.plot_wireframe(X,Y,array[frame])
        ax.plot_wireframe(Y,X,base, color='grey')
        ax.plot_surface(X,Y,array[frame], cmap='jet')
        #ax.plot_wireframe(Y,X,base, color='grey')
        ax.set_xlabel('x-axis')
        ax.set_ylabel('y-axis')
        ax.set_zlabel('displacement')
        ax.set_zlim(-0.5,0.5)
        return [ax]
        #return graph                    

    # Most important line, this is what actually handles the animation by calling the `animate' function with a frame number (integer)
    ani = FuncAnimation(fig, animate, init_func=init, blit=True, frames=1000, interval=20, repeat=True)   # Code to create animations
    
    if(save_animation):                                  # If you wish to save the animation, do so with the dpi and fps set here 
        ani.save(save_name, dpi=save_dpi, fps=save_fps)

    return ani                                           # When calling this FuncAnimation function from with a code, all the information that update the window 
                                                         # are attributes of the object ani. If you do not keep a reference to it around, then ani is garbage collected 
                                                         # all information about the graphs disappears when calling from within a function. You don't need this when you call 
                                                         # aren't calling the animation from within a function.

In [ ]:
%matplotlib notebook
animate_3d(u_t)

In [ ]:
def amplitude(u_t, a, b):
    u = []
    for i in range(0, 250):
        u.append(u_t[i][a][b])
    return u

# for i in range(0, 1000):
#     u.append(u_t[i][50][20])
#plt.plot(amplitude(u_t, 10, 10), color='red', linewidth=2)
#plt.plot(amplitude(u_t, 20, 20), color='red', linewidth=2)
#plt.plot(amplitude(u_t, 30, 30), color='red', linewidth=2)
#plt.plot(amplitude(u_t, 40, 40), color='red', linewidth=2)
#plt.plot(amplitude(u_t, 50, 50), color='red', linewidth=2)
plt.plot(amplitude(u_t, 50, 10), color='red', linewidth=2)
#plt.plot(amplitude(u_t, 20, 15), color='red', linewidth=2)
plt.xlabel('time index')
plt.ylabel('amplitude')
plt.xlim(5,)